In [4]:
import os
import sys

import numpy as np
import pandas as pd
import openpyxl

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill

In [17]:
os.getcwd() # current working directory

'C:\\Users\\ENURI\\Desktop\\김동훈\\Works\\Andrew\\Python'

In [334]:
df = pd.read_excel('./Data/20221225_네이버절대매칭.xlsx', index_col = 0, sheet_name = 'MAT_RATE')
df2 = pd.read_excel('./Data/20221225_네이버절대매칭.xlsx', index_col = 0, sheet_name = 'ADD_EP_CNT')

In [431]:
df_1 = pd.read_excel('./Data/20221226_화장품몰_절대매칭.xlsx', sheet_name = 0)
df_2 = pd.read_excel('./Data/20221226_화장품몰_절대매칭.xlsx', sheet_name = 1)
df_3 = pd.read_excel('./Data/20221226_화장품몰_절대매칭.xlsx', sheet_name = 2)

In [335]:
df['cate'] = df['cate'].apply(lambda x: '0' + str(x) if len(str(x)) % 2 != 0 else str(x))
df = df.sort_values(['rate']).reset_index(drop = True)

df2['cate'] = df2['cate'].apply(lambda x: '0' + str(x) if len(str(x)) % 2 != 0 else str(x))

### 엑셀 파일 생성

In [186]:
'''
>>> wb = openpyxl.Workbook()
엑셀 파일 형태의 객체(인스턴스)를 만들어서 wb라는 변수로 선언
해당 객체를 생성하는 순간 'Sheet'라는 이름을 가진 기본 시트가 생성된다.

만약 기존에 있는 엑셀 파일을 읽어서 작업을 하고싶은 경우라면,
>>> wb = openpyxl.load_workbook('file_name.xlsx')

이후,
>>> wb.create_sheet('sheet_name', 'sheet_loc')
라는 메서드를 이용해서 추가적인 시트를 생성할 수 있지만, 기본적으로 생성된 Sheet는 남아있다.
sheet_loc 값을 입력해주지 않았을 경우에는 맨 뒤에 생성된다
'''

wb = openpyxl.Workbook()

wb.create_sheet('test_sheet1', 0)
wb.create_sheet('test_sheet2')

print(wb.sheetnames)
print(wb.get_sheet_names())

['test_sheet1', 'Sheet', 'test_sheet2']
['test_sheet1', 'Sheet', 'test_sheet2']


C:\Users\ENURI\AppData\Local\Temp\ipykernel_22936\2353242651.py:21: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  print(wb.get_sheet_names())


### Sheet 활성화

In [187]:
'''
작업은 sheet 단위로 이루어지며,
기본적으로 sheet 객체를 변수에 저장하여 작업한다.

active 메서드를 이용하거나 원하는 시트명을 선언하여 활성화하며,
create_sheet 메서드 사용시에 변수 선언을 하면, 해당 변수에서 해당 시트가 활성화되어 저장된다.

active를 이용할 시에는,
>>> ws1 = wb.active

시트명을 선언할 경우,
>>> ws2 = wb['sheet_name']

로 실행하며, active 할 경우 가장 첫 번째 시트가 활성화된다.
'''

ws1 = wb.active
ws2 = wb['Sheet']
ws3 = wb['test_sheet2']

print(ws1, ws2, ws3)

<Worksheet "test_sheet1"> <Worksheet "Sheet"> <Worksheet "test_sheet2">


### 시트명 변경

In [188]:
'''
각 시트별로 변경해주어야 한다.

>>> ws.title

해당 변수에 저장되어있는 시트의 시트명을 불러오며, 해당 값에 다른 값을 저장해주면 시트명이 변경된다.
>>> ws.title = '변경 시트명'
'''

print(wb.sheetnames)

for i, s in enumerate(wb.sheetnames):
    print(wb[s])
    wb[s].title = f'테스트_시트{i+1}'
    
print(wb.sheetnames)

['test_sheet1', 'Sheet', 'test_sheet2']
<Worksheet "test_sheet1">
<Worksheet "Sheet">
<Worksheet "test_sheet2">
['테스트_시트1', '테스트_시트2', '테스트_시트3']


### 기본 사용 방법 (Pandas to openpyxl)

In [273]:
'''
기본적으로, 판다스 데이터 프레임에 있는 값을 row 단위로 워크시트에 append 하는 식으로 데이터를 처리한다

dataframe_to_rows 라는 함수 사용
>>> dataframe_to_rows(df, index = True, header = True)

해당 함수의 결과로 generator 가 생성되며, 해당 결과를 for문을 통해 워크 시트에 추가해준다.
index, header 의 경우, 데이터프레임의 인덱스 값과 컬럼 값을 엑셀 데이터에 포함할 것인지 여부를 묻는 것이다.
유의할 점은, index를 True로 했을 경우, 컬럼 rows와 데이터 rows 사이에 공백 행이 하나 생긴다.

[None, 'cate', 'lcate_name', 'mcate_name', 'rate', 'price']
[None]
[0, 801, '뷰티', '스킨케어', 0.2854984894259819, 154006632]

for 문을 통해 구성 값을 워크시트로 추가했을 경우,
cell 단위로 데이터들이 저장되며, 원하는 cell에 접근하는 방법은,

컬럼명 선언 후, 해당 값의 위치 index 입력 - 컬럼명은 엑셀 기준으로, 알파벳으로 구성된다.
ws['A'][0] - 엑셀 위치 상으로 'A'열의 1행 값 / 데이터 프레임 기준으로 1열 1행 값 (컬럼 포함일 경우, 첫 번째 컬럼 명)
ws[1][0] - 엑셀 위치  상으로 1행의 'A'열 값 / 데이터 프레임 기준으로 1열 1행 값 (컬럼 포함일 경우, 첫 번째 컬럼 명)

기본적으로 다양한 엑셀 상에서의 조건을 적용할 때, 셀 단위로 접근하여 해당 셀에 옵션 값을 부여하는 식이다.
'''

wb = openpyxl.Workbook()
ws = wb.active
ws.title = '테스트_시트1'

for row in dataframe_to_rows(df, index = False, header = True):
    ws.append(row)

In [291]:
row

['1662', '생활,취미', '기타동물용품', 1.0, 5041540]

In [274]:
# 해당 셀에 테두리 및 볼드 처리 (판다스의 컬럼, 인덱스에 적용되는 효과)
for cell in ws[1]:
    cell.style = 'Pandas'

In [275]:
# 열 너비 맞춤 - 데이터 값에 상관없이 기본 셀 크기로 저장이 되는데 이를 조정하기 위한 방법
columns = None
margin = 2

for i, column_cells in enumerate(ws.columns):
    is_ok = False
    if columns == None:
        is_ok = True
    elif isinstance(columns, list) and i in columns:
        is_ok = True
        
    if is_ok:
        val_list = [str(cell.value) for cell in column_cells]
        if re.search('[ㄱ-ㅎ가-힣]+', val_list[1]):
            length = max([len(x) for x in val_list]) * 2
        else:
            length = max([len(x) for x in val_list])

        ws.column_dimensions[column_cells[0].column_letter].width = length + margin

In [276]:
from openpyxl.styles import PatternFill

In [287]:
# 색 칠하기

fill_color = PatternFill(start_color = 'FFF2CC',
                         end_color = 'FFF2CC',
                         fill_type = 'solid')

for i in df.loc[df['rate'] < 0.5].index:
    for cell in ws[i + 2]:
        cell.fill = fill_color

In [288]:
wb.save('test.xlsx')

### 개별 셀에 직접 값 입력

In [423]:
wb = openpyxl.Workbook()
ws = wb.active
ws.title = '테스트_시트1'

### 여러 데이터 프레임을 하나의 시트에 작성하고 싶을 때

각 셀 위치에 직접 접근해서 깂을 입력한다.

여러 개의 테이블을 하나의 시트에 입력하고 싶다면, 해당 데이터의 개수 등을 참조하여

원하는 간격만큼 데이터를 띄워서 입력하는 식으로 로직을 구성하면 된다.

In [424]:
start_col = 2
start_row = 2

for i, col in enumerate(df.columns):
    ws.cell(start_row, start_col + i).value = col
    ws.cell(start_row, start_col + i).style = 'Pandas'
    
last_col = start_col + i
    
for r, rows in enumerate(df.values):
    for c, cols in enumerate(rows):
        ws.cell(start_row + 1 + r, start_col + c).value = df.values[r][c]
        
        if rows[3] < 0.5:
            ws.cell(start_row + 1 + r, start_col + c).fill = fill_color

In [425]:
column_gap = 2
start_col = last_col + column_gap + 1
start_row = 2

for i, col in enumerate(df2.columns):
    ws.cell(start_row, start_col + i).value = col
    ws.cell(start_row, start_col + i).style = 'Pandas'
    
last_col = start_col + i

for r, rows in enumerate(df2.values):
    for c, cols in enumerate(rows):
        ws.cell(start_row + 1 + r, start_col + c).value = df2.values[r][c]
        
ws.cell(start_row + 1 + r + 1, start_col).value = '총합'
# ws.cell(start_row + 1 + r + 1, start_col).style = 'Pandas'

ws.cell(start_row + 1 + r + 1, start_col + len(rows) - 1).value = df2['add_ep_cnt'].sum()
# ws.cell(start_row + 1 + r + 1, start_col + len(rows) - 1).style = 'Pandas'

In [426]:
# 열 너비 맞춤 - 데이터 값에 상관없이 기본 셀 크기로 저장이 되는데 이를 조정하기 위한 방법
columns = None
margin = 2

for i, column_cells in enumerate(ws.columns):
    is_ok = False
    if columns == None:
        is_ok = True
    elif isinstance(columns, list) and i in columns:
        is_ok = True
        
    if is_ok:
        val_list = [str(cell.value) for cell in column_cells]
        if re.search('[ㄱ-ㅎ가-힣]+', val_list[2]):
            length = max([len(x) for x in val_list]) * 2
        else:
            length = max([len(x) for x in val_list])

        ws.column_dimensions[column_cells[0].column_letter].width = length + margin

In [475]:
# Naver / Kakao / Coupang
def output_type1(mat_rate_df, add_pl_list_df, site):
    mat_rate_df = mat_rate_df.sort_values(['rate'], ascending = True)
    
    if site == 'naver':
        mat_rate = 0.5
    elif site == 'kakao':
        mat_rate = 0.5
    elif site == 'coupang':
        mat_rate = 0.7
    else:
        pass
    
    wb = openpyxl.Workbook()

    ws_1 = wb.active
    ws_1.title = 'DB별 현황'
    ws_2 = wb.create_sheet(datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d'))

    # mat_rate_df
    start_col = 2
    start_row = 2
    fill_color = PatternFill(start_color = 'FFF2CC',
                             end_color = 'FFF2CC',
                             fill_type = 'solid')

    for i, col in enumerate(mat_rate_df.columns):
        ws_2.cell(start_row, start_col + i).value = col
        ws_2.cell(start_row, start_col + i).style = 'Pandas'
        
    last_col = start_col + i
    
    for r, rows in enumerate(mat_rate_df.values):
        for c, val in enumerate(rows):
            ws_2.cell(start_row + 1 + r, start_col + c).value = val
            
            if rows[3] < mat_rate:
                ws_2.cell(start_row + 1 + r, start_col + c).fill = fill_color
                
    # add_pl_list_df
    column_gap = 2
    start_col = last_col + column_gap + 1
    start_row = 2
    
    for i, col in enumerate(add_pl_list_df.columns):
        ws_2.cell(start_row, start_col + i).value = col
        ws_2.cell(start_row, start_col + i).style = 'Pandas'
        
    last_col = start_col + i
    
    for r, rows in enumerate(add_pl_list_df.values):
        for c, val in enumerate(rows):
            ws_2.cell(start_row + 1 + r, start_col + c).value = val
            
    ws_2.cell(start_row + 1 + r + 1, start_col).value = '총합'
    ws_2.cell(start_row + 1 + r + 1, start_col + len(rows) - 1).value = add_pl_list_df['add_ep_cnt'].sum()
    
    # 셀 너비
    columns = None
    margin = 2

    ws_1 = fit_column_size(ws_1, columns = None, margin = 2)
    ws_2 = fit_column_size(ws_2, columns = None, margin = 2)
    
    return wb

def output_type2(mat_rate_df, add_pl_cnt_df, add_pl_list_df):
    mat_rate_df = mat_rate_df.sort_values(['차이'], ascending = False)
    
    wb = openpyxl.Workbook()

    ws = wb.active
    ws.title = datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d')
    
    # mat_rate_df
    start_col = 2
    start_row = 2
    fill_color = PatternFill(start_color = 'FFF2CC',
                             end_color = 'FFF2CC',
                             fill_type = 'solid')

    for i, col in enumerate(mat_rate_df.columns):
        ws.cell(start_row, start_col + i).value = col
        ws.cell(start_row, start_col + i).style = 'Pandas'
        
    last_col = start_col + i
    
    for r, rows in enumerate(mat_rate_df.values):
        for c, val in enumerate(rows):
            ws.cell(start_row + 1 + r, start_col + c).value = val
            
            if rows[-2] > 0:
                ws.cell(start_row + 1 + r, start_col + c).fill = fill_color
                
    last_row = start_row + r + 1
                
    # add_pl_cnt_df
    row_gap = 1
    start_col = 2
    start_row = last_row + row_gap + 1
    
    for i, col in enumerate(add_pl_cnt_df.columns):
        ws.cell(start_row, start_col + i).value = col
        ws.cell(start_row, start_col + i).style = 'Pandas'
        
    for r, rows in enumerate(add_pl_cnt_df.values):
        for c, val in enumerate(rows):
            ws.cell(start_row + 1 + r, start_col + c).value = val
            
    last_row = start_row + r
    
    # add_pl_list_df
    col_gap = 1
    start_col = last_col + col_gap + 1
    start_row = 2
    
    for i, col in enumerate(add_pl_list_df.columns):
        ws.cell(start_row, start_col + i).value = col
        ws.cell(start_row, start_col + i).style = 'Pandas'
        
    last_col = start_col + i
    
    for r, rows in enumerate(add_pl_list_df.values):
        for c, val in enumerate(rows):
            ws.cell(start_row + 1 + r, start_col + c).value = val
            
    ws = fit_column_size(ws, columns = None, margin = 2)
    
    return wb

def fit_column_size(worksheet, columns = None, margin = 2):
    for i, column_cells in enumerate(worksheet.columns):
        is_ok = False
        if columns == None:
            is_ok = True
        elif isinstance(columns, list) and i in columns:
            is_ok = True

        if is_ok:
            val_list = [str(cell.value) for cell in column_cells]
            length = max([len(x) for x in val_list]) * 1.5

            worksheet.column_dimensions[column_cells[0].column_letter].width = length + margin
            
    return worksheet

In [476]:
wb_1 = output_type1(df, df2, 'naver')
wb_2 = output_type2(df_1, df_2, df_3)

In [479]:
wb_1.save('test1.xlsx')

In [480]:
wb_2.save('test2.xlsx')